<a href="https://colab.research.google.com/github/dude123studios/AdvancedDeepLearning/blob/main/Estimator_for_FashionMnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import time
import tensorflow as tf
import numpy as np

LABEL_DIM = 10

(train_img,train_lbs),(test_img,test_lbs) = tf.keras.datasets.fashion_mnist.load_data()
TRAINING_SIZE = len(train_img)
TEST_SIZE = len(test_img)

train_img = np.asarray(train_img,dtype='float32')/255.0
train_img = train_img.reshape((TRAINING_SIZE,28,28,1))
test_img = np.asarray(test_img,dtype='float32')/255.0
test_img = test_img.reshape((TEST_SIZE,28,28,1))

train_lbs = tf.keras.utils.to_categorical(train_lbs,LABEL_DIM)
test_lbs = tf.keras.utils.to_categorical(test_lbs,LABEL_DIM)
train_lbs = train_lbs.astype('float32')
test_lbs = test_lbs.astype('float32')
print(test_lbs.shape)
print(train_lbs.shape)

4423680/4422102 [==============================] - 0s 0us/step
(10000, 10)
(60000, 10)


In [3]:
inputs = tf.keras.layers.Input(shape=(train_img.shape[1:]))
x = tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2)(x)
x = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu')(x)
x = tf.keras.layers.MaxPool2D((2,2),strides=2)(x)
x = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64,activation='relu')(x)
preds = tf.keras.layers.Dense(LABEL_DIM,activation='softmax')(x)
model = tf.keras.models.Model(inputs=inputs,outputs=preds)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0     

In [4]:
optim = tf.keras.optimizers.SGD()
model.compile(optimizer=optim,loss='categorical_crossentropy',metrics=['accuracy'])
strategy = tf.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)
estimator = tf.keras.estimator.model_to_estimator(model,config=config)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Using the Keras model provided.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmprpxq358k', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7fadb03c68d0>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_distribute_coordinator_mode': N

In [5]:
def input_fn(images,labels,epochs,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((images,labels))
    SHUFFLE_SIZE = 60000
    dataset = dataset.shuffle(SHUFFLE_SIZE).repeat(epochs).batch(batch_size)
    dataset = dataset.prefetch(None)
    return dataset

BATCH_SIZE = 512
EPOCHS = 50
estimator_train_results=\
    estimator.train(input_fn=lambda:input_fn(train_img,train_lbs,EPOCHS,BATCH_SIZE))
print(estimator_train_results)

estimator.evaluate(lambda:input_fn(test_img,test_lbs,1,BATCH_SIZE))

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CP

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


INFO:tensorflow:Inference Time : 0.41340s
INFO:tensorflow:Finished evaluation at 2021-01-10-21:51:06
INFO:tensorflow:Saving dict for global step 5860: accuracy = 0.8289, global_step = 5860, loss = 0.4741744
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5860: /tmp/tmprpxq358k/model.ckpt-5860


{'accuracy': 0.8289, 'global_step': 5860, 'loss': 0.4741744}